In [100]:
import random

import matplotlib.pyplot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js
from math import sin, cos, pi

TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax

    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))

    def add_line(self, event):
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()

    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)

    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()

In [2]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs

    def add_points(self, points):
        self.points = self.points + points

class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs

    def add(self, line):
        self.lines.append(line)

    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]],
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]])
                           for scene in js.loads(json)]

    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]

    def add_scene(self, scene):
        self.scenes.append(scene)

    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points],
                          "lines":[linesCol.lines for linesCol in scene.lines]}
                         for scene in self.scenes])

    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None

    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None

    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None

    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None

    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()

# Rozwiązanie Lab2

## Generowanie zbiorów punktów

### Zbiór punktów A

In [3]:
%matplotlib notebook

points_a = [(random.uniform(-100, 100), random.uniform(-100, 100)) for _ in range(100)]
plot_a = Plot(points = [PointsCollection(points_a)])
plot_a.draw()

<IPython.core.display.Javascript object>

### Zbiór punktów B

In [4]:
%matplotlib notebook

def circle_point(t):
    return 10*cos(2*pi*t), 10*sin(2*pi*t)
points_b = [circle_point(random.uniform(0, 1)) for _ in range(100)]
plot_b = Plot(points = [PointsCollection(points_b)])
plot_b.draw()

<IPython.core.display.Javascript object>

### Zbiór punktów C

In [5]:
%matplotlib notebook

def rectangle_point():
    side = random.randrange(4)
    t = random.uniform(-10, 10)
    if side == 0:
        return t, -10
    elif side == 1:
        return 10, t
    elif side == 3:
        return t, 10
    else:
        return -10, t

points_c = [rectangle_point() for _ in range(100)]
plot_c = Plot(points = [PointsCollection(points_c)])
plot_c.draw()

<IPython.core.display.Javascript object>

### Zbiór punktów D

In [6]:
%matplotlib notebook

def diagonal1(t):
    return t, t
def diagonal2(t):
    return t, 10 - t

points_d = [(0.0, 0.0), (10, 0), (10, 10), (0, 10)] + [(0, random.uniform(0, 10)) for _ in range(25)] + [(random.uniform(0, 10), 0) for _ in range(25)] + [diagonal1(random.uniform(0, 10)) for _ in range(20)] + [diagonal2(random.uniform(0, 10)) for _ in range(20)]
plot_d = Plot(points = [PointsCollection(points_d)])
plot_d.draw()

<IPython.core.display.Javascript object>

## Funkcje generujące dowolne zbiory punktów

### Zbiór n punktów ze współrzędnymi z przedziału [a, b]

In [7]:
%matplotlib notebook
def generate_a(n, a, b):
    return [(random.uniform(a, b), random.uniform(a, b)) for _ in range(n)]

Plot(points=[PointsCollection(generate_a(1000, -100, 100))]).draw()

<IPython.core.display.Javascript object>

### Zbiór n punktów na okręgu o promieniu r i środku (x, y)

In [8]:
%matplotlib notebook

def generate_b(n, r, x, y):
    def circle_p(t):
        return r*cos(2*pi*t) + x, r*sin(2*pi*t) + y
    return [circle_p(random.uniform(0, 1)) for _ in range(n)]

Plot(points=[PointsCollection(generate_b(100, 100, 35, 25))]).draw()

<IPython.core.display.Javascript object>

### Zbiór n punktów, na prostokącie o wierzchołkach (x1, y1)(lewy dolny) i (x2, y2)(prawy górny)

In [9]:
%matplotlib notebook

def generate_c(n, x1, y1, x2, y2):
    side_a = x2 - x1
    side_b = y2 - y1
    perimeter = 2*side_a + 2*side_b
    def rectangle_p(t):
        x = side_a
        if t <= x:
            return x1 + t, y1
        x += side_b
        if t <= x:
            return x2, y1 + x - t
        x += side_a
        if t <= x:
            return x2 + t - x, y2
        x += side_b
        return x1, y2 + t - x
    return [rectangle_p(random.uniform(0, perimeter)) for _ in range(n)]

Plot(points=[PointsCollection(generate_c(100, 5, 6, 10, 10))]).draw()

<IPython.core.display.Javascript object>

### Zbiór punktów leżących na bokach i przekątnych kwadratu o lewym dolnym wierzchołku w (x1, y1) i boku długości side

In [10]:
%matplotlib notebook

def generate_d(n_side, n_diag, side, x1, y1):
    def diag1(t):
        return t, t - x1 + y1
    def diag2(t):
        return t, y1 + x1 + side - t
    return [(x1, y1), (x1 + side, y1), (x1, y1 + side), (x1 + side, y1 + side)] + [(x1, random.uniform(y1, y1 + side)) for _ in range(n_side)] + [(random.uniform(x1, x1 + side), y1) for _ in range(n_side)] + [diag1(random.uniform(x1, x1 + side)) for _ in range(n_diag)] + [diag2(random.uniform(x1, x1 + side)) for _ in range(n_diag)]

Plot(points=[PointsCollection(generate_d(30, 30, 10, 2, 5))]).draw()

<IPython.core.display.Javascript object>

## Implementacja funkcji obliczającej wyznacznik macierzy

In [11]:
epsilon = pow(10, -10)
def det(a, b, c):
    d =  (a[0]*b[1] + a[1]*c[0] + b[0]*c[1] -
            c[0]*b[1] - a[0]*c[1] - b[0]*a[1])
    if abs(d) < epsilon:
        return 0
    return d

def distance(p1, p2): # Jako, że funkcja służy tylko do porównania, który z dwóch punktów jest bliżej trzeciego, nie ma sensu używanie pierwiastka
        return (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2

def cmp(point, a, b):
    d = det(point, a, b)
    if d == 0:
        return distance(point, a) < distance(point, b)
    return d < 0

## Implementacje algorytmów do znajdowania otoczki wypukłej( wersje z wizualizacją poszczególnych kroków)

### Implementacja Algorytmu Jarvisa

In [34]:
def add_to_scenes1(scenes, points, convex_hull, lines):
    scenes.append(Scene(points=[PointsCollection(points, color='blue'), PointsCollection(convex_hull, color='red')], lines=[LinesCollection(lines, color="red")]))


def jarvis_visualise(points, filename=""): # Drugi argument to nazwa pliku do którego zostanie zapisany wynikowy zbiór punktów należących do otoczki, Jeśli sie go nie poda to wynik nie zostanie zapisany do pliku
    scenes = [Scene(points=[PointsCollection(points, color="blue")])]
    n = len(points)
    lower = 0
    for i in range(1, n):
        if points[lower][1] > points[i][1] or (points[lower][1] == points[i][1] and points[lower][0] > points[i][0]):
            lower = i
    visit = [False for _ in range(n)]
    lines = []
    convex_hull = [points[lower]]
    add_to_scenes1(scenes, points, convex_hull[:], lines[:])
    current = lower
    while not visit[current]:
        visit[current] = True
        follow = -1
        for i in range(n):
            if i != current and not visit[i]:
                if follow == -1 or cmp(points[current], points[follow], points[i]):
                    follow = i
        if current != lower and det(points[current], points[follow], points[lower]) <= 0:
            follow = lower
        convex_hull.append(points[follow])
        lines.append((points[current], points[follow]))
        add_to_scenes1(scenes, points, convex_hull[:], lines[:])
        current = follow

    convex_hull.pop()
    if filename:
        f = open(filename, 'w')
        for a, b in convex_hull:
            f.write("(" + str(a) + ", " + str(b) + ")\n")
        f.close()
    return scenes


### Jarvis dla zbioru A

In [38]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(points_a))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zbioru B

In [39]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(points_b))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zbioru C

In [40]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(points_c))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zbioru D

In [42]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(points_d))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zmodyfikowanego zbioru A

In [46]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(generate_a(1000, 0, 1000)))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zmodyfikowanego zbioru B

In [45]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(generate_b(100, 1000, 500, 500)))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zmodyfikowanego zbioru C

In [48]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(generate_c(300, -100, -50, 100, 50)))
plot.draw()

<IPython.core.display.Javascript object>

### Jarvis dla zmodyfikowanego zbioru D

In [49]:
%matplotlib notebook

plot = Plot(scenes=jarvis_visualise(generate_d(30, 20, 50, 0, 0)))
plot.draw()

<IPython.core.display.Javascript object>

### Implementacja Algorytmu Grahama

In [32]:
from functools import cmp_to_key

def add_to_scenes2(scenes, points, convex_hull, highlight_point, highlight_lines, lines):
    scenes.append(Scene(points=[PointsCollection(points, color='blue'), PointsCollection(convex_hull, color='red'), PointsCollection(highlight_point, color="orange")], lines=[LinesCollection(lines, color="red"), LinesCollection(highlight_lines, color="orange")]))


def remove_duplicate_angles(point, p1):
    p2 = [p1[0]]
    for x in p1[1:]:
        if det(point, p2[-1], x):
            p2.append(x)
    return p2



def graham_visualise(points, filename=""): # Tak samo jak w jarvisie, przy podaniu drugiego argumentu, wierzchołki otoczki zapisują się w pliku o tej nazwie
    p = points[:]
    scenes = [Scene(points=[PointsCollection(points, color="blue")])]
    n = len(p)
    lower = 0
    for i in range(1, n):
        if p[lower][1] > p[i][1] or (p[lower][1] == p[i][1] and p[lower][0] > p[i][0]):
            lower = i

    point = p[lower]
    p.pop(lower)
    def comparator(a, b):
        d = det(point, a, b)
        if d == 0:
            return distance(point, b) - distance(point, a)
        return -d

    p = sorted(p, key=cmp_to_key(comparator))
    p = remove_duplicate_angles(point, p)
    lines = []
    add_to_scenes2(scenes, points, [], [point], [], lines[:])
    convex_hull = [point, p[0]]
    lines.append((point, p[0]))
    add_to_scenes2(scenes, points, convex_hull[:], [p[0]], [], lines[:])
    p.append(point)
    for x in p[1:]:
        add_to_scenes2(scenes, points, convex_hull[:], [x], [(convex_hull[-1], x)], lines[:])
        while cmp(convex_hull[-2], convex_hull[-1], x):
            add_to_scenes2(scenes, points, convex_hull[:], [x], [], lines[:])
            convex_hull.pop()
            lines.pop()
            add_to_scenes2(scenes, points, convex_hull[:], [x], [], lines[:])
            add_to_scenes2(scenes, points, convex_hull[:], [x], [(convex_hull[-1], x)], lines[:])

        lines.append((convex_hull[-1], x))
        convex_hull.append(x)
        add_to_scenes2(scenes, points, convex_hull[:], [], [], lines[:])

    convex_hull.pop()
    if filename:
        f = open(filename, 'w')
        for a, b in convex_hull:
            f.write("(" + str(a) + ", " + str(b) + ")\n")
        f.close()

    return scenes



### Graham dla zbioru A

In [50]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(points_a))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zbioru B

In [51]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(points_b))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zbioru C

In [52]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(points_c))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zbioru D

In [53]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(points_d))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zmodyfikowanego zbioru A

In [55]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(generate_a(1000, 0, 1000)))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zmodyfikowanego zbioru B

In [56]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(generate_b(100, 1000, 500, 500)))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zmodyfikowanego zbioru C

In [57]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(generate_c(300, -100, -50, 100, 50)))
plot.draw()

<IPython.core.display.Javascript object>

### Graham dla zmodyfikowanego zbioru D

In [58]:
%matplotlib notebook

plot = Plot(scenes=graham_visualise(generate_d(30, 20, 50, 0, 0)))
plot.draw()

<IPython.core.display.Javascript object>

## Implementacje algorytmów ( wersje bez wizualizacji, do zmierzenia czasu działania obu algorytmów)

### Implementacja algorytmu Jarvisa

In [108]:
def jarvis_time(points):
    n = len(points)
    lower = 0
    for i in range(1, n):
        if points[lower][1] > points[i][1] or (points[lower][1] == points[i][1] and points[lower][0] > points[i][0]):
            lower = i

    visit = [False for _ in range(n)]
    convex_hull = [points[lower]]
    current = lower
    while not visit[current]:
        visit[current] = True
        follow = -1
        for i in range(n):
            if i != current and not visit[i]:
                if follow == -1 or cmp(points[current], points[follow], points[i]):
                    follow = i
        if current != lower and det(points[current], points[follow], points[lower]) <= 0:
            follow = lower
        convex_hull.append(points[follow])
        current = follow

    convex_hull.pop()
    return convex_hull


### Implementacja algorytmu Grahama

In [110]:
def graham_time(points):
    p = points[:]
    n = len(p)
    lower = 0
    for i in range(1, n):
        if p[lower][1] > p[i][1] or (p[lower][1] == p[i][1] and p[lower][0] > p[i][0]):
            lower = i

    point = p[lower]
    p.pop(lower)
    def comparator(a, b):
        d = det(point, a, b)
        if d == 0:
            return distance(point, b) - distance(point, a)
        return -d

    p = sorted(p, key=cmp_to_key(comparator))
    p = remove_duplicate_angles(point, p)
    convex_hull = [point, p[0]]
    p.append(point)
    for x in p[1:]:
        while cmp(convex_hull[-2], convex_hull[-1], x):
            convex_hull.pop()
        convex_hull.append(x)

    convex_hull.pop()
    return convex_hull

### Porównanie czasu działania na zbiorze typu A

In [132]:
from time import time

number_of_points = [100, 1000, 2500, 5000, 10000, 20000]

for n in number_of_points:
    print("Ilość punktów: ", n)
    test_points = generate_a(n, 0, 10*n)
    t1 = time()
    graham_result = graham_time(test_points)
    t2 = time()
    print("Czas algorytmu Grahama: ", t2 - t1)
    t1 = time()
    jarvis_result = jarvis_time(test_points)
    t2 = time()
    print("Czas algorytmu Jarvisa: ", t2 - t1)
    print(graham_result == jarvis_result)

Ilość punktów:  100
Czas algorytmu Grahama:  0.0
Czas algorytmu Jarvisa:  0.0
True
Ilość punktów:  1000
Czas algorytmu Grahama:  0.015625476837158203
Czas algorytmu Jarvisa:  0.04865312576293945
True
Ilość punktów:  2500
Czas algorytmu Grahama:  0.04015827178955078
Czas algorytmu Jarvisa:  0.05198931694030762
True
Ilość punktów:  5000
Czas algorytmu Grahama:  0.05582118034362793
Czas algorytmu Jarvisa:  0.0937497615814209
True
Ilość punktów:  10000
Czas algorytmu Grahama:  0.1275017261505127
Czas algorytmu Jarvisa:  0.23572254180908203
True
Ilość punktów:  20000
Czas algorytmu Grahama:  0.35376644134521484
Czas algorytmu Jarvisa:  0.8174502849578857
True


<IPython.core.display.Javascript object>

In [143]:
from time import time

number_of_points = [100, 1000, 2500, 5000, 10000, 20000]

for n in number_of_points:
    print("Ilość punktów: ", n)
    test_points = generate_b(n, 10*n, 0, 0)
    t1 = time()
    graham_result = graham_time(test_points)
    t2 = time()
    print("Czas algorytmu Grahama: ", t2 - t1)
    if n < 10000:
        t1 = time()
        jarvis_result = jarvis_time(test_points)
        t2 = time()
        print("Czas algorytmu Jarvisa: ", t2 - t1)
        print(graham_result == jarvis_result)

Ilość punktów:  100
Czas algorytmu Grahama:  0.002998828887939453
Czas algorytmu Jarvisa:  0.0049991607666015625
True
Ilość punktów:  1000
Czas algorytmu Grahama:  0.019994258880615234
Czas algorytmu Jarvisa:  0.5659475326538086
True
Ilość punktów:  2500
Czas algorytmu Grahama:  0.03124856948852539
Czas algorytmu Jarvisa:  3.996742010116577
True
Ilość punktów:  5000
Czas algorytmu Grahama:  0.046878814697265625
Czas algorytmu Jarvisa:  12.499260902404785
True
Ilość punktów:  10000
Czas algorytmu Grahama:  0.31316709518432617
Ilość punktów:  20000
Czas algorytmu Grahama:  0.3129298686981201


<IPython.core.display.Javascript object>

In [151]:
from time import time

number_of_points = [100, 1000, 2500, 5000, 10000, 20000]

for n in number_of_points:
    print("Ilość punktów: ", n)
    test_points = generate_c(n, 0, 0, 10*n, 20*n)
    t1 = time()
    graham_result = graham_time(test_points)
    t2 = time()
    print("Czas algorytmu Grahama: ", t2 - t1)
    t1 = time()
    jarvis_result = jarvis_time(test_points)
    t2 = time()
    print("Czas algorytmu Jarvisa: ", t2 - t1)
    print(graham_result == jarvis_result)

Ilość punktów:  100
Czas algorytmu Grahama:  0.0010001659393310547
Czas algorytmu Jarvisa:  0.00099945068359375
False
Ilość punktów:  1000
Czas algorytmu Grahama:  0.014995336532592773
Czas algorytmu Jarvisa:  0.03199052810668945
False
Ilość punktów:  2500
Czas algorytmu Grahama:  0.05898165702819824
Czas algorytmu Jarvisa:  0.021993160247802734
False
Ilość punktów:  5000
Czas algorytmu Grahama:  0.09697270393371582
Czas algorytmu Jarvisa:  0.06597900390625
False
Ilość punktów:  10000
Czas algorytmu Grahama:  0.17193174362182617
Czas algorytmu Jarvisa:  0.10776686668395996
False
Ilość punktów:  20000
Czas algorytmu Grahama:  0.5525600910186768
Czas algorytmu Jarvisa:  0.4036121368408203
False


<IPython.core.display.Javascript object>

In [153]:
from time import time

number_of_points = [100, 1000, 2500, 5000, 10000, 20000]

for n in number_of_points:
    print("Ilość punktów: ", n)
    test_points = generate_d(n // 4, n // 4, 10*n, 0, 0)
    t1 = time()
    graham_result = graham_time(test_points)
    t2 = time()
    print("Czas algorytmu Grahama: ", t2 - t1)
    t1 = time()
    jarvis_result = jarvis_time(test_points)
    t2 = time()
    print("Czas algorytmu Jarvisa: ", t2 - t1)
    print(graham_result == jarvis_result)


Ilość punktów:  100
Czas algorytmu Grahama:  0.015606880187988281
Czas algorytmu Jarvisa:  0.0
True
Ilość punktów:  1000
Czas algorytmu Grahama:  0.015623092651367188
Czas algorytmu Jarvisa:  0.015624523162841797
True
Ilość punktów:  2500
Czas algorytmu Grahama:  0.07847356796264648
Czas algorytmu Jarvisa:  0.015994548797607422
True
Ilość punktów:  5000
Czas algorytmu Grahama:  0.12400674819946289
Czas algorytmu Jarvisa:  0.04398798942565918
True
Ilość punktów:  10000
Czas algorytmu Grahama:  0.21894311904907227
Czas algorytmu Jarvisa:  0.0625157356262207
True
Ilość punktów:  20000
Czas algorytmu Grahama:  0.5933568477630615
Czas algorytmu Jarvisa:  0.10936760902404785
True
